In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
import hyperopt
from hyperopt import fmin, tpe, Trials, space_eval, hp
import warnings
warnings.filterwarnings(action='ignore')

# HyperOpt

In [2]:
data = pd.read_csv('/home/tigran/Downloads/code/Data/StateFarm_missing.csv', sep=';')
data.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1),
                                                   data['Response'],
                                                   test_size=0.3,
                                                   stratify=data['Response'],
                                                   random_state=42)

In [4]:
cat_columns = X_train.select_dtypes(include='object').columns.tolist()
num_columns = X_train.select_dtypes(exclude='object').columns.tolist()

In [5]:
num_pipe = Pipeline([
    ('imp', SimpleImputer())
])

cat_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [6]:
transformers = [('num', num_pipe, num_columns),
               ('cat', cat_pipe, cat_columns)]

In [7]:
transformer = ColumnTransformer(transformers=transformers)

In [8]:
pipe = Pipeline([
    ('tr', transformer),
    ('boost', GradientBoostingClassifier(random_state=42))
])

In [9]:
initial_pipe = clone(pipe)
pipe

Pipeline(steps=[('tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer())]),
                                                  ['Customer Lifetime Value',
                                                   'Income',
                                                   'Monthly Premium Auto',
                                                   'Months Since Last Claim',
                                                   'Months Since Policy '
                                                   'Inception',
                                                   'Number of Open Complaints',
                                                   'Number of Policies']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer()),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Coverage', 'Education',
                                                   'EmploymentStatus',
                                                   'Gender'])])),
                ('boost', GradientBoostingClassifier(random_state=42))])

In [10]:
def hyperopt_objective(params):
    pipe.set_params(**params)
    return -cross_val_score(pipe, X_train, y_train, cv=5).mean()

In [11]:
space_hyperopt = {
    'boost__learning_rate': hp.loguniform('boost__learning_rate', 0.001, 0.1),
    'tr__num__imp__strategy': hp.choice('tr__num__imp__strategy', ['mean', 'median', 'constant']),
    'tr__cat__imp__strategy': hp.choice('tr__cat__imp__strategy', ['most_frequent', 'constant'])
}

In [12]:
trials = Trials()

In [13]:
best = fmin(fn=hyperopt_objective,
           space=space_hyperopt,
           algo=tpe.suggest,
           max_evals=20,
           trials=trials,
           rstate=np.random.default_rng(123))

100%|████████| 20/20 [00:49<00:00,  2.50s/trial, best loss: -0.9050818260120586]


In [14]:
print('best: ', best)

best:  {'boost__learning_rate': 1.0062813714974137e+00, 'tr__cat__imp__strategy': 0, 'tr__num__imp__strategy': 1}


In [15]:
pipe

Pipeline(steps=[('tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Customer Lifetime Value',
                                                   'Income',
                                                   'Monthly Premium Auto',
                                                   'Months Since Last Claim',
                                                   'Months Since Policy '
                                                   'Inception',
                                                   'Number of Open Complaints',
                                                   'Number of Policies']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Coverage', 'Education',
                                                   'EmploymentStatus',
                                                   'Gender'])])),
                ('boost',
                 GradientBoostingClassifier(learning_rate=1.0686988258771697,
                                            random_state=42))])

In [16]:
trials.trials[19]

{'state': 2,
 'tid': 19,
 'spec': None,
 'result': {'loss': -0.8993970714900948, 'status': 'ok'},
 'misc': {'tid': 19,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'boost__learning_rate': [19],
   'tr__cat__imp__strategy': [19],
   'tr__num__imp__strategy': [19]},
  'vals': {'boost__learning_rate': [1.0686988258771697],
   'tr__cat__imp__strategy': [0],
   'tr__num__imp__strategy': [1]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2023, 5, 14, 14, 33, 32, 159000),
 'refresh_time': datetime.datetime(2023, 5, 14, 14, 33, 34, 664000)}

In [17]:
def accuracy_test(X_tr, y_tr, X_tst, y_tst, opt_results, space=None, optimize='optuna'):
    
    if space is None and optimize == 'hyperopt':
        raise ValueError('Set search space')
    if space is not None and optimize == 'optuna':
        raise ValueError('Search space setting only for hyperopt')
        
    if optimize == 'hyperopt':
        opt_results = space_eval(space, opt_results)
    
    pipe.set_params(**opt_results)
    pipe.fit(X_tr, y_tr)
    
    test_predict = pipe.predict(X_tst)
    return accuracy_score(y_tst, test_predict)

In [18]:
accuracy_test(X_train, y_train, X_test, y_test, best, space_hyperopt, 'hyperopt')

0.9083601286173634

# Scikit-Optimize

In [19]:
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args

In [20]:
space_skopt = {
    'boost__learning_rate': Real(0.001, 0.1, prior='uniform'),
    'boost__max_depth': Integer(2, 8),
    'tr__num__imp__strategy': Categorical(['mean', 'median', 'constant']),
    'tr__cat__imp__strategy': Categorical(['most_frequent', 'constant'])
}

In [21]:
pipe = clone(initial_pipe)

In [22]:
opt = BayesSearchCV(pipe, space_skopt, n_iter=10, scoring='accuracy', random_state=42, cv=5)
opt.fit(X_train, y_train)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('tr',
                                         ColumnTransformer(transformers=[('num',
                                                                          Pipeline(steps=[('imp',
                                                                                           SimpleImputer())]),
                                                                          ['Customer '
                                                                           'Lifetime '
                                                                           'Value',
                                                                           'Income',
                                                                           'Monthly '
                                                                           'Premium '
                                                                           'Auto',
                                                                           'Months '
                                                                           'Since '
                                                                           'Last '
                                                                           'Claim',
                                                                           'Months '
                                                                           'Since '
                                                                           'Policy '
                                                                           'Inception',
                                                                           'Number '
                                                                           'of '
                                                                           'Open '
                                                                           'Complaints',
                                                                           'Number '
                                                                           'of '
                                                                           'Policies']),
                                                                         ('cat',
                                                                          Pipeline(steps=[('imp',
                                                                                           Simpl...
              search_spaces={'boost__learning_rate': Real(low=0.001, high=0.1, prior='uniform', transform='normalize'),
                             'boost__max_depth': Integer(low=2, high=8, prior='uniform', transform='normalize'),
                             'tr__cat__imp__strategy': Categorical(categories=('most_frequent', 'constant'), prior=None),
                             'tr__num__imp__strategy': Categorical(categories=('mean', 'median', 'constant'), prior=None)})

In [23]:
print(opt.best_score_)
print(json.loads(json.dumps(opt.best_params_)))
print(opt.score(X_test, y_test))

9.236864771748493e-01
{'boost__learning_rate': 0.07366877378057127, 'boost__max_depth': 8, 'tr__cat__imp__strategy': 'constant', 'tr__num__imp__strategy': 'constant'}
0.9368971061093248


In [24]:
results = pd.DataFrame(opt.cv_results_)
results.columns = results.columns.str.replace('param_', '')
tbl = results.pivot_table(values=['mean_test_score'],
                         index=['boost__learning_rate',
                               'boost__max_depth',
                               'tr__cat__imp__strategy',
                               'tr__num__imp__strategy'])

tbl = tbl.sort_values(by='mean_test_score', ascending=False)
tbl

,,,,mean_test_score
boost__learning_rate,boost__max_depth,tr__cat__imp__strategy,tr__num__imp__strategy,
0.073669,8,constant,constant,0.923686
0.083901,7,constant,median,0.920930
0.054797,8,constant,median,0.919552
0.045038,8,constant,mean,0.917657
0.095592,6,most_frequent,mean,0.917485
0.062091,7,constant,median,0.916968
0.080156,5,most_frequent,mean,0.907666
0.041600,6,most_frequent,mean,0.907321
0.081427,3,most_frequent,median,0.901292


In [25]:
pipe = clone(initial_pipe)

In [26]:
space_skopt2 = [
    Real(0.001, 0.1, name='boost__learning_rate', prior='uniform'),
    Integer(2, 8, name='boost__max_depth'),
    Categorical(['mean', 'median', 'constant'], name='tr__num__imp__strategy'),
    Categorical(['most_frequent', 'constant'], name='tr__cat__imp__strategy')
]

In [27]:
@use_named_args(space_skopt2)
def objective(**params):
    pipe.set_params(**params)
    return -np.mean(cross_val_score(pipe, 
                                   X_train,
                                   y_train, 
                                   cv=5,
                                   n_jobs=-1))

In [28]:
res = gp_minimize(objective, space_skopt2, n_initial_points=5, n_calls=10, random_state=42)

/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 

/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/tigran/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 

In [29]:
print(np.abs(res.fun))

9.305770887166236e-01


In [30]:
pd.DataFrame({'boost__learning_rate': pd.Series(res.x[0]),
             'boost__max_depth': pd.Series(res.x[1]),
             'tr__num__imp__strategy': pd.Series(res.x[2]),
             'tr__cat__imp__strategy': pd.Series(res.x[3])})

,boost__learning_rate,boost__max_depth,tr__num__imp__strategy,tr__cat__imp__strategy
0,0.099916,8,mean,constant


# Optuna

In [31]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, CategoricalDistribution, UniformDistribution, FloatDistribution

In [20]:
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x-2)**2

In [36]:
sampler = TPESampler(seed=10)

In [22]:
study = optuna.create_study(sampler=sampler)

In [64]:
study.optimize(objective, n_trials=100)

[I 2023-05-14 17:40:13,741] Trial 0 finished with value: 11.740305123732652 and parameters: {'x': 5.426412865334919}. Best is trial 0 with value: 11.740305123732652.
[I 2023-05-14 17:40:13,743] Trial 1 finished with value: 134.21132166837336 and parameters: {'x': -9.58496101281197}. Best is trial 0 with value: 11.740305123732652.
[I 2023-05-14 17:40:13,743] Trial 2 finished with value: 0.45288148546152784 and parameters: {'x': 2.672964698525508}. Best is trial 2 with value: 0.45288148546152784.
[I 2023-05-14 17:40:13,744] Trial 3 finished with value: 8.857038183425992 and parameters: {'x': 4.976077650772236}. Best is trial 2 with value: 0.45288148546152784.
[I 2023-05-14 17:40:13,745] Trial 4 finished with value: 4.120330620698611 and parameters: {'x': -0.02985975394819107}. Best is trial 2 with value: 0.45288148546152784.
[I 2023-05-14 17:40:13,745] Trial 5 finished with value: 56.311022881961755 and parameters: {'x': -5.504067089383047}. Best is trial 2 with value: 0.4528814854615278

[I 2023-05-14 17:40:13,873] Trial 45 finished with value: 3.144344161954048 and parameters: {'x': 3.7732298672067444}. Best is trial 41 with value: 0.03732512911138656.
[I 2023-05-14 17:40:13,878] Trial 46 finished with value: 1.5911977066776877 and parameters: {'x': 0.7385731465210963}. Best is trial 41 with value: 0.03732512911138656.
[I 2023-05-14 17:40:13,882] Trial 47 finished with value: 0.11945767886741451 and parameters: {'x': 2.345626501974913}. Best is trial 41 with value: 0.03732512911138656.
[I 2023-05-14 17:40:13,885] Trial 48 finished with value: 5.3176862671402265 and parameters: {'x': -0.30601089918071}. Best is trial 41 with value: 0.03732512911138656.
[I 2023-05-14 17:40:13,888] Trial 49 finished with value: 1.7864426669984759 and parameters: {'x': 3.3365787170976784}. Best is trial 41 with value: 0.03732512911138656.
[I 2023-05-14 17:40:13,891] Trial 50 finished with value: 0.04627537358283107 and parameters: {'x': 2.2151171159690253}. Best is trial 41 with value: 0.

[I 2023-05-14 17:40:14,006] Trial 90 finished with value: 2.3190616643509827 and parameters: {'x': 0.4771534337462022}. Best is trial 72 with value: 3.0803586568735782e-06.
[I 2023-05-14 17:40:14,009] Trial 91 finished with value: 0.0006583480349128822 and parameters: {'x': 1.9743417063133013}. Best is trial 72 with value: 3.0803586568735782e-06.
[I 2023-05-14 17:40:14,013] Trial 92 finished with value: 0.041922544876724255 and parameters: {'x': 2.204749956963913}. Best is trial 72 with value: 3.0803586568735782e-06.
[I 2023-05-14 17:40:14,017] Trial 93 finished with value: 0.4178128608667105 and parameters: {'x': 2.646384452834929}. Best is trial 72 with value: 3.0803586568735782e-06.
[I 2023-05-14 17:40:14,020] Trial 94 finished with value: 2.303623726082213 and parameters: {'x': 3.517769325715279}. Best is trial 72 with value: 3.0803586568735782e-06.
[I 2023-05-14 17:40:14,023] Trial 95 finished with value: 4.83152446255749 and parameters: {'x': 4.198072897462113}. Best is trial 72 

In [65]:
print(study.best_params)

{'x': 1.998244904943636}


In [66]:
print(study.best_value)

3.0803586568735782e-06


In [67]:
print(study.best_trial)

FrozenTrial(number=72, state=TrialState.COMPLETE, values=[3.0803586568735782e-06], datetime_start=datetime.datetime(2023, 5, 14, 17, 40, 13, 950089), datetime_complete=datetime.datetime(2023, 5, 14, 17, 40, 13, 952396), params={'x': 1.998244904943636}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'x': FloatDistribution(high=10.0, log=False, low=-10.0, step=None)}, trial_id=72, value=None)


In [47]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [55]:
def objective(trial):
    params = {
        'boost__learning_rate': trial.suggest_uniform('boost__learning_rate', 0.001, 0.1),
        'boost__max_depth': trial.suggest_int('boost__max_depth', 2, 8),
        'tr__num__imp__strategy': trial.suggest_categorical('tr__num__imp__strategy', ['mean', 'median', 'constant']),
        'tr__cat__imp__strategy': trial.suggest_categorical('tr__cat__imp__strategy', ['most_frequent', 'constant'])
    }
    pipe.set_params(**params)
    return np.mean(cross_val_score(pipe, X_train, y_train, cv=kfold))

In [49]:
study = optuna.create_study(sampler=sampler, direction='maximize')
optuna.logging.disable_default_handler()
study.optimize(objective, n_trials=10)

[I 2023-05-14 18:49:53,272] A new study created in memory with name: no-name-ff970425-0c59-42a3-85cf-39bdf36886c8


In [50]:
best = study.best_trial

In [51]:
print(best.value)

0.9286821705426357


In [52]:
for key, value in best.params.items():
    print('{}: {}'.format(key, value))

boost__learning_rate: 0.07656081812722443
boost__max_depth: 8
tr__num__imp__strategy: median
tr__cat__imp__strategy: most_frequent


In [56]:
accuracy_test(X_train, y_train, X_test, y_test, best.params, space=None, optimize='optuna')

0.932475884244373

In [32]:
from lightgbm import LGBMClassifier

In [33]:
lgbm_pipe = Pipeline([
    ('tr', transformer),
    ('lgboost', LGBMClassifier(random_state=42, n_estimators=200))
])

In [34]:
def lgbm_objective(trial):
    params = {
        'lgboost__learning_rate': trial.suggest_uniform('lgboost__learning_rate', 0.001, 0.1),
        'lgboost__max_depth': trial.suggest_int('lgboost__max_depth', 2, 8),
        'tr__num__imp__strategy': trial.suggest_categorical('tr__num__imp__strategy', ['mean', 'median', 'constant']),
        'tr__cat__imp__strategy': trial.suggest_categorical('tr__cat__imp__strategy', ['most_frequent', 'constant'])
    }
    lgbm_pipe.set_params(**params)
    return np.mean(cross_val_score(lgbm_pipe, X_train, y_train, cv=5))

In [37]:
study = optuna.create_study(sampler=sampler, direction='maximize')

[I 2023-05-14 18:42:15,201] A new study created in memory with name: no-name-e2258e0a-fdb6-446f-8722-5a6bc8f64146


In [38]:
study.optimize(lgbm_objective, n_trials=10)

[I 2023-05-14 18:42:47,469] Trial 0 finished with value: 0.9000861326442722 and parameters: {'lgboost__learning_rate': 0.07736074368340785, 'lgboost__max_depth': 2, 'tr__num__imp__strategy': 'median', 'tr__cat__imp__strategy': 'most_frequent'}. Best is trial 0 with value: 0.9000861326442722.
[I 2023-05-14 18:42:47,725] Trial 1 finished with value: 0.9035314384151594 and parameters: {'lgboost__learning_rate': 0.07629254050769692, 'lgboost__max_depth': 3, 'tr__num__imp__strategy': 'constant', 'tr__cat__imp__strategy': 'constant'}. Best is trial 1 with value: 0.9035314384151594.
[I 2023-05-14 18:42:48,178] Trial 2 finished with value: 0.9250645994832041 and parameters: {'lgboost__learning_rate': 0.08144947520355925, 'lgboost__max_depth': 6, 'tr__num__imp__strategy': 'constant', 'tr__cat__imp__strategy': 'most_frequent'}. Best is trial 2 with value: 0.9250645994832041.
[I 2023-05-14 18:42:48,493] Trial 3 finished with value: 0.9007751937984496 and parameters: {'lgboost__learning_rate': 0.0

In [39]:
print(study.best_value)

0.9250645994832041


In [40]:
study.best_params

{'lgboost__learning_rate': 0.08144947520355925,
 'lgboost__max_depth': 6,
 'tr__num__imp__strategy': 'constant',
 'tr__cat__imp__strategy': 'most_frequent'}

In [41]:
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping

In [42]:
features = X_train.copy()
cols = list(features.columns[features.dtypes == 'object'])
for c in cols:
    features[c] = features[c].astype('category').cat.codes
labels = np.where(y_train == 'Yes', 1, 0)
dtrain = lgb.Dataset(data=features, label=labels)

In [43]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    'boosting_type': 'gbdt'
}

In [44]:
tuner = lgb.LightGBMTunerCV(params, 
                           dtrain,
                           num_boost_round=300,
                           folds=KFold(n_splits=3, shuffle=True, random_state=42),
                           callbacks=[early_stopping(30)])

[I 2023-05-14 18:47:48,824] A new study created in memory with name: no-name-443ce345-00a2-49c8-aeb1-7240c99b7278


In [45]:
tuner.run()

feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 30 rounds


feature_fraction, val_score: 0.895778:  14%|8     | 1/7 [00:00<00:02,  2.90it/s]

Early stopping, best iteration is:
[98]	cv_agg's auc: 0.895778 + 0.0151915


feature_fraction, val_score: 0.896784:  29%|#7    | 2/7 [00:00<00:01,  3.20it/s]

...

Early stopping, best iteration is:
[38]	cv_agg's auc: 0.902769 + 0.0234389


min_data_in_leaf, val_score: 0.902769:  60%|###6  | 3/5 [00:01<00:00,  2.51it/s]

Early stopping, best iteration is:
[38]	cv_agg's auc: 0.902769 + 0.0234389


min_data_in_leaf, val_score: 0.902769: 100%|######| 5/5 [00:01<00:00,  3.00it/s]

Early stopping, best iteration is:
[38]	cv_agg's auc: 0.902769 + 0.0234389
Early stopping, best iteration is:
[38]	cv_agg's auc: 0.902769 + 0.0234389


In [46]:
print(tuner.best_score)
print(tuner.best_params)

for key, value in tuner.best_params.items():
    print('  {}: {}'.format(key, value))

0.902768787566469
{'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 116, 'feature_fraction': 0.7, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
  objective: binary
  metric: auc
  verbosity: -1
  boosting_type: gbdt
  feature_pre_filter: False
  lambda_l1: 0.0
  lambda_l2: 0.0
  num_leaves: 116
  feature_fraction: 0.7
  bagging_fraction: 1.0
  bagging_freq: 0
  min_child_samples: 20
